# Assignment15.3

In [10]:
import tensorflow as tf
print(tf.__version__)

2.5.0


In [11]:
import cv2
import random
import numpy as np
import scipy.misc
from PIL import Image

In [12]:
# Class for an target object in a game
class Target:
    def __init__(self, x, y):
        self.x = x
        self.y = y
        self.width = 0
        self.height = 0
        self.speed = (0,1)
        self.active = True

    def getDimensions(self):
        return (self.x, self.y, self.width, self.height)

    def centerOrigin(self):
        return (self.x - self.width/2, self.y - self.height/2)

    def update(self):
        self.x += self.speed[0]
        self.y += self.speed[1]

In [13]:
# Count the number of pixels hitting targets
def hit_value(image, target):
    tmp = np.zeros((int(height), int(width), 1), np.uint8)
    rect = target.getDimensions()
    cv2.rectangle(tmp, (rect[0], rect[1]), (rect[2], rect[3]), (255), -1)
    num = np.sum(cv2.bitwise_and(image,tmp))/255
    print(num)
    return num

In [14]:
# Create multiple targets randomly
def create_targets(count, dim):
    targets = list()
    for i in range(count):
        # Random x location
        tgt = Target(random.randint(0, frame_size[0]-dim[0]), 0)
        tgt.width = dim[0]
        tgt.height = dim[1]
        targets.append(tgt)

    return targets

In [15]:
cd G:/handtracking/handtracking-master

G:\handtracking\handtracking-master


In [16]:
from utils import detector_utils as detector_utils
import cv2
import tensorflow as tf
import datetime
import argparse

In [17]:
detection_graph, sess = detector_utils.load_inference_graph()

> ====== loading HAND frozen graph into memory
>  ====== Hand Inference graph loaded.


In [18]:
#from keras.preprocessing.image import load_img
#from tensorflow.keras.utils import load_img
#from keras.preprocessing.image import img_to_array
#from keras.preprocessing.image import array_to_img

# Create windows to show the captured images
cv2.namedWindow ("window_motion", cv2.WINDOW_AUTOSIZE )
cv2.namedWindow ("window_scene", cv2.WINDOW_AUTOSIZE )

# Structuring element for morphological operator
es = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(9,9))

# Webcam settings
cap = cv2.VideoCapture(0)
width  = cap.get(cv2.CAP_PROP_FRAME_WIDTH)  
height = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)
frame_size = (int(width),int(height))

# Video settings
# Set to True if you want to record a video of the game
writeVideo = True
# Video format
fourcc = cv2.VideoWriter_fourcc('M','J','P','G') 
fps = 30
# Create a video file
if writeVideo:
    video_writer = cv2.VideoWriter("C:/Users/78089/anaconda3/pics/game_record_YifanLi.avi", fourcc, fps, frame_size)

# Prepare processing images
previous = np.zeros((int(height), int(width), 3), np.uint8)
difference = np.zeros((int(height), int(width), 3), np.uint8)
current = np.zeros((int(height), int(width), 3), np.uint8)

# Load a target object picture
bola_original = cv2.imread('C:/Users/78089/anaconda3/pics/Aqua-Ball-Red-icon.png')
# Resize image
dim = (64, 64)
bola = cv2.resize(bola_original, dim, interpolation = cv2.INTER_AREA)
# Load a target object's mask picture
mask_original = cv2.imread('C:/Users/78089/anaconda3/pics/input-mask.png',0)
# Resize image
mask = cv2.resize(mask_original, dim, interpolation = cv2.INTER_AREA)

# Number of targets
nbolas = 9
targets = create_targets(nbolas, dim)
# Delay before the game begin
initialDelay = 100
# Score begins with 0
score = 0
# -------------- hand detection ===========
num_hands_detect = 2
score_thresh=0.2
im_width, im_height = (cap.get(3), cap.get(4))
# -------------- hand detection ===========

# While get the frame from webcam
while True:
    # Capture frame-by-frame
    ret, capture = cap.read()
    capture = cv2.flip(capture, 1)
    scene = capture.copy() 
    
    # -------------- motion detection ===========
    # Difference between frames
    #current = cv2.blur(capture,(15,15))
    #difference = cv2.absdiff(current, previous)
    # -------------- motion detection ===========
    # -------------- hand detection ===========
    image_np = cv2.cvtColor(capture, cv2.COLOR_BGR2RGB)
    difference = np.zeros_like(image_np)
    boxes, scores = detector_utils.detect_objects(image_np, detection_graph, sess)
    #detector_utils.draw_box_on_image(num_hands_detect, score_thresh,
    #                                     scores, boxes, im_width, im_height,
    #                                     difference)
    for i in range(num_hands_detect):
        if (scores[i] > score_thresh):
            (left, right, top, bottom) = (boxes[i][1] * im_width, boxes[i][3] * im_width,
                                          boxes[i][0] * im_height, boxes[i][2] * im_height)
            p1 = (int(left), int(top))
            p2 = (int(right), int(bottom))
            cv2.rectangle(difference, p1, p2, (255, 255, 255), -1)
            
    detector_utils.draw_box_on_image(num_hands_detect, score_thresh,
                                         scores, boxes, im_width, im_height,
                                         scene)
    # -------------- hand detection ===========

    frame = cv2.cvtColor(difference, cv2.COLOR_BGR2GRAY)
    ret,frame = cv2.threshold(frame,10,255,cv2.THRESH_BINARY)
    kernel = np.ones((3,3),np.uint8)
    frame = cv2.dilate(frame,kernel,iterations = 1)
    
    if initialDelay <= 0:
        for t in targets:
            if t.active:
                nzero = hit_value(frame, t)
                # Check whether the movement part hit the target objects
                if nzero < 100:
                    print(nzero)
                    # Draws the target to screen
                    roi = t.getDimensions()
                    current_img = Image.fromarray(scene)
                    bola_img = Image.fromarray(bola)
                    mask_img = Image.fromarray(mask)
                    current_img.paste(bola_img, (t.x, t.y), mask_img)
                    scene = np.asarray(current_img)

                    t.update()
                
                else:
                    t.y = 0
                    t.x = random.randint(0, frame_size[0]-dim[0])
                    if t.speed[1] < 15:
                        t.speed = (0, t.speed[1]+1)
                    score += nbolas
                
                # If the target hits the bottom
                if t.y + t.height >= frame_size[1]:
                    t.active = False
                    # Reduce the number of target balls
                    nbolas -= 1
                    # Reduce scores
                    score -= nbolas


    # Initialize font setting
    fontface = cv2.FONT_HERSHEY_SIMPLEX
    fontscale = 1
    fontcolor = (255, 255, 255)
    text = "Score: " + str(score)
    cv2.putText(scene,text,(10,frame_size[1]-10),fontface,fontscale,fontcolor,1)
    cv2.imshow("window_motion", frame) 
    cv2.imshow("window_scene", scene)
    if writeVideo:
         video_writer.write(scene)
    
    x = cv2.waitKey(25)
    if x==27:
        break;
    
    previous = current.copy()
    initialDelay -= 1

cap.release()
cv2.destroyAllWindows()


0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
